In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 4,
  "metadata": {
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python",
      "version": "3.9"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Ejemplo de experimentación con MLflow\n",
        "Este notebook entrena un modelo sencillo (RandomForest) y lo registra en MLflow.\n",
        "\n",
        "Lee variables de entorno definidas en `paths.env`, usando `os.getenv`."
      ]
    },
    {
      "cell_type": "code",
      "metadata": {},
      "execution_count": null,
      "outputs": [],
      "source": [
        "import os\n",
        "import mlflow\n",
        "import mlflow.sklearn\n",
        "from sklearn.datasets import load_iris\n",
        "from sklearn.model_selection import train_test_split\n",
        "from sklearn.ensemble import RandomForestClassifier\n",
        "\n",
        "# Tomamos HOST y PORT desde environment variables (paths.env)\n",
        "mlflow_host = os.getenv(\"MLFLOW_HOST\", \"localhost\")\n",
        "mlflow_port = os.getenv(\"MLFLOW_PORT\", \"5000\")\n",
        "\n",
        "# Construimos la URL del servidor MLflow\n",
        "tracking_uri = f\"http://{mlflow_host}:{mlflow_port}\"\n",
        "mlflow.set_tracking_uri(tracking_uri)\n",
        "\n",
        "print(f\"Usando MLflow tracking URI: {tracking_uri}\")\n",
        "\n",
        "# Nombre de experimento\n",
        "experiment_name = \"G4_4toTaller_Experiment\"\n",
        "mlflow.set_experiment(experiment_name)\n",
        "\n",
        "# Carga dataset de ejemplo\n",
        "iris = load_iris()\n",
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    iris.data, iris.target, test_size=0.2, random_state=42\n",
        ")\n",
        "\n",
        "# Listas de hiperparámetros\n",
        "n_estimators_list = [10, 50, 100, 150]\n",
        "max_depth_list = [2, 5, 10, None]\n",
        "\n",
        "for n_est in n_estimators_list:\n",
        "    for md in max_depth_list:\n",
        "        with mlflow.start_run():\n",
        "            mlflow.log_param(\"n_estimators\", n_est)\n",
        "            mlflow.log_param(\"max_depth\", md)\n",
        "\n",
        "            clf = RandomForestClassifier(\n",
        "                n_estimators=n_est,\n",
        "                max_depth=md,\n",
        "                random_state=42\n",
        "            )\n",
        "            clf.fit(X_train, y_train)\n",
        "\n",
        "            acc = clf.score(X_test, y_test)\n",
        "            mlflow.log_metric(\"accuracy\", acc)\n",
        "\n",
        "            # Registrar modelo\n",
        "            mlflow.sklearn.log_model(clf, artifact_path=\"model\")\n",
        "\n",
        "print(\"¡Experimentos finalizados! Revisa MLflow para ver los resultados.\")"
      ]
    }
  ]
}